In [6]:
from keras.layers import Embedding, ELU, Dropout, Flatten, Input, Dense, BatchNormalization, Conv1D, MaxPooling1D, LSTM, CuDNNLSTM
from keras.layers import concatenate, multiply, Bidirectional
from keras.models import Sequential
from keras.layers import TimeDistributed, Reshape, RepeatVector, Lambda, Activation
from keras.regularizers import l2, l1_l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras import Model
from keras.callbacks import LearningRateScheduler

import pandas as pd
import numpy as np

data = pd.read_csv('train.csv',header=None,dtype='float',names=list(range(4096)))
test = pd.read_csv('test.csv',header=None,dtype='float',names=list(range(4096)))
label = pd.read_csv('train_label.csv')['category'].values.reshape(-1, 1)

In [10]:
maxlen=4096
embed_size=64
lstm_layer_size = 100
num_layers = 1

model = Sequential()
model.add(Embedding(256, embed_size, input_length=maxlen, embeddings_regularizer=l2(1e-4)))
model.add(Dropout(0.5))

for l in range(0, num_layers - 1):
    model.add(Bidirectional(CuDNNLSTM(lstm_layer_size, kernel_regularizer=l2(1e-5), recurrent_regularizer=l2(1e-5), return_sequences=True)))
    model.add(Dropout(0.5))
model.add(Bidirectional(CuDNNLSTM(lstm_layer_size, kernel_regularizer=l2(1e-5), recurrent_regularizer=l2(1e-5), return_sequences=True)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid', name='loss_out'))

def scheduler(epoch):
    if epoch%1==0 and epoch!=0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr*.5)
        print("lr changed to {}".format(lr*.5))
    return K.get_value(model.optimizer.lr)

lr_decay = LearningRateScheduler(scheduler)

optimizer = Adam(lr=0.004, clipnorm=1.0)
model.compile(optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 4096, 64)          16384     
_________________________________________________________________
dropout_11 (Dropout)         (None, 4096, 64)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 4096, 200)         132800    
_________________________________________________________________
dropout_12 (Dropout)         (None, 4096, 200)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 819200)            0         
_________________________________________________________________
loss_out (Dense)             (None, 1)                 819201    
Total params: 968,385
Trainable params: 968,385
Non-trainable params: 0
_________________________________________________________________


In [8]:
es = EarlyStopping(monitor='binary_accuracy', min_delta=0, patience=0, verbose=0, mode='auto')
model.fit(data, label, batch_size=200, epochs=10, verbose=1, callbacks=[es, lr_decay])
result = model.predict(test, batch_size=None, verbose=1)
prediction = pd.DataFrame({'malware': result[:,0]})
prediction.to_csv('prediction_rnn.csv', index=True,header=True)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[200,4096,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: embedding_3/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_3/embeddings/read, embedding_3/Cast)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics_2/binary_accuracy/Mean_1/_317 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1920_metrics_2/binary_accuracy/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to R